In [ ]:
#!pip install "pillow<7"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import torch 

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
import os

In [ ]:
torch.cuda.is_available()

In [ ]:
valid_ratio = 0.2
transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                transforms.RandomAffine(degrees=10,scale=(1,1.5)),
                                transforms.RandomRotation(15, fill=(0,)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))
                               ])



trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)

num_train = len(trainset)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_ratio*num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# Data samplers for train and validation sets
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

trainloader = torch.utils.data.DataLoader(trainset, sampler=train_sampler, batch_size=32, shuffle=False)
validloader = torch.utils.data.DataLoader(trainset, sampler=valid_sampler, batch_size=32, shuffle=False)
testloader = torch.utils.data.DataLoader(testset,batch_size=64, shuffle = False)

In [ ]:
len(testset)

In [ ]:
dataiter = iter(trainloader)
images, label = dataiter.next()
images = images.numpy()

In [ ]:
fig = plt.figure(figsize=(25,6))
for idx in range(16):
    ax = fig.add_subplot(2,20/2,idx+1,xticks=[],yticks=[])
    ax.imshow(np.squeeze(images[idx]),cmap='gray')
    # .item() gets the value contained in a Tensor
    ax.set_title(str(label[idx].item()))

In [ ]:
Label_list = ['0','1','2','3','4','5','6','7','8','9']

In [ ]:
images[0].shape

In [ ]:
plt.imshow(images[0].squeeze())


In [ ]:
from torch import nn, optim
import torch.nn.functional as F

In [ ]:
64*14*14


In [ ]:
class ConvMNIST(nn.Module):
    def __init__(self):
        super(ConvMNIST, self).__init__()  # Indicates that the MNIST class inherits properties from nn.Module.
        
        self.conv1 = nn.Conv2d(1,32,3,1,1)      # 28*28*1
        self.conv2 = nn.Conv2d(32,64,3,1,1) 
        self.pool = nn.MaxPool2d(kernel_size=2) # 14*14*32
        self.fc2 = nn.Linear(64*14*14,1000)
        self.fc3 = nn.Linear(1000,256)
        self.fc4 = nn.Linear(256,10)
        self.dropout = nn.Dropout(p=0.4)
        
    def forward(self,x):
        
        x = self.dropout(F.relu(self.conv1(x)))
        x = self.dropout(F.relu(self.conv2(x)))
        x = self.pool(x)
        x = x.view(-1,64*14*14)
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = F.log_softmax(self.fc4(x),dim=1)
        
        return x

In [ ]:
model = ConvMNIST()
criterion = nn.NLLLoss()
optimiser = optim.Adam(model.parameters(),lr=0.001)

In [ ]:
model

In [ ]:
#os.listdir()

In [ ]:
epochs = 30
train_losses = []
valid_losses = []
valid_loss_min = np.inf
accuracy_list = []

for e in range(epochs):
    print('Starting epoch:',e)
    batch_loss = 0
    valid_loss = 0
    for images, labels in trainloader:
        output = model(images)
        loss = criterion(output,labels)
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        
        batch_loss += loss.item()
    with torch.no_grad():
        model.eval()
        for images, labels in validloader:
            output = model(images)
            loss = criterion(output,labels)
            valid_loss +=loss.item()
            
            probs = torch.exp(output)
            _, classes = torch.topk(probs, k=1, dim=1)
            equals = classes == labels.view(*classes.shape)
            accuracy = torch.mean(equals.type(torch.FloatTensor))
            
    train_losses.append(batch_loss/len(trainloader))
    valid_losses.append(valid_loss/len(validloader))
    print(f"Training loss: {batch_loss/len(trainloader)}, Validation loss: {valid_loss/len(validloader)}")
    print(f"Accuracy (Validation): {accuracy.item()*100}%")
    accuracy_list.append(accuracy.item())
    if (valid_loss/len(validloader)) <= valid_loss_min:
        print(f"Validation loss decreased from {valid_loss_min} to {valid_loss/len(validloader)}. Saving the model")
        valid_loss_min = valid_loss/len(validloader)
        torch.save(model,'mnist_model/model_'+str(e)+'_loss'+str(valid_loss/len(validloader))+'.pth')
    model.train()

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(valid_losses, label='Validation loss')
plt.plot(accuracy_list, label='Validation accuracy')
plt.legend(frameon=False)

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

test_loss = 0
accuracy = 0
for inputs, labels in testloader:
    #inputs = inputs.view(inputs.shape[0],-1)
    
    output = model(inputs)
    loss = criterion(output,labels)
    
    test_loss +=loss.item()
            
    probs = torch.exp(output)
    _, classes = torch.topk(probs, k=1, dim=1)
    equals = classes == labels.view(*classes.shape)
    accuracy += torch.mean(equals.type(torch.FloatTensor))
    for i in range(len(labels)):
        label = labels.data[i]
        class_correct[label] += equals[i].item()
        class_total[label] += 1
        
test_loss = test_loss/len(testloader)
print(f"Test Loss: {test_loss}, Accuracy: {(accuracy/len(testloader))*100}%")
print(class_correct)
for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

In [ ]:
# Passing a single image
data = iter(testloader)
img, label = data.next()
img = img[2].unsqueeze(dim=0)

out = model(img)

prob = torch.exp(out)
value, index = torch.topk(prob, k=1,dim=1)

plt.imshow(img.squeeze())
print(f"Predicted class {Label_list[index[0][0]]}")
print(label[2])
print(index[0][0])